In [1]:
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
import pandas as pd 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics.pairwise import linear_kernel,cosine_similarity

In [2]:
scaler=MinMaxScaler()
#content similarity
content_features=pd.read_csv('./data/content_features.csv',index_col='anime_id')
f_df=pd.read_csv("./data/temp.csv",index_col=0)
content_df=pd.read_csv('./data/content_based_data.csv',index_col='anime_id')

content_df.fillna("unknown",inplace=True)
cf_index=pd.read_csv('./data/cf_index.csv',index_col=0)
pop_anime=list(cf_index['anime_id'])

cv=TfidfVectorizer(stop_words='english')
vectors=cv.fit_transform(content_features['tags']).toarray()

categorical=f_df.drop(['anime_id', 'url', 'title','episodes','aired', 'duration', 'score', 'scored_by', 'rank',
       'popularity', 'synopsis', 'year','genres', 'themes','image_url', 'small_image_url', 'large_image_url',
       'embed_url', 'yt_image'],axis=1)
onehot_encoder = OneHotEncoder(handle_unknown='ignore')
onehot_encoded = onehot_encoder.fit_transform(categorical)
weight_tfidf = 0.85
weight_onehot = 0.15
combined_features = hstack([weight_tfidf*vectors, weight_onehot*onehot_encoded])
cosine_sim_combined = linear_kernel(combined_features, combined_features)
cosine_sim_combined=scaler.fit_transform(cosine_sim_combined)

# collaborative similarity
item_factors = np.loadtxt('./data/items.txt', dtype=float)
cf_similarity=cosine_similarity(item_factors)
cf_similarity=scaler.fit_transform(cf_similarity)

In [6]:
cosine_sim_combined

array([[1.        , 0.46401908, 0.2133589 , ..., 0.14461369, 0.23134134,
        0.12110306],
       [0.47740334, 1.        , 0.19946137, ..., 0.13828609, 0.22668007,
        0.15732515],
       [0.24346644, 0.21038287, 1.        , ..., 0.21358544, 0.20906669,
        0.11930875],
       ...,
       [0.14810113, 0.1198199 , 0.18562258, ..., 1.        , 0.16359089,
        0.22699741],
       [0.23003012, 0.20552157, 0.17618725, ..., 0.15873422, 1.        ,
        0.09888035],
       [0.13997779, 0.15430367, 0.1039261 , ..., 0.2405015 , 0.11973392,
        1.        ]])

In [8]:
comp=cosine_sim_combined.astype(np.float16)

In [9]:
npz_file = 'content_sim_compressed.npz'
np.savez_compressed(npz_file, array=comp)

In [15]:
comp_cf=cf_similarity.astype(np.float16)
npz_file = 'cf_sim_compressed.npz'
np.savez_compressed(npz_file, array=comp_cf)

In [13]:
# import numpy as np
import joblib

# Create a 6029x6029 array with random data and convert to float16
# array = np.random.rand(6029, 6029).astype(np.float16)

# Save the array using joblib with compression
joblib_file = 'con1_compressed.joblib'
joblib.dump(comp, joblib_file, compress=9)  # Higher values for more compression (3 is a good balance)

print(f"Array saved to {joblib_file}")

Array saved to con1_compressed.joblib


In [14]:
# npz_file = 'array_compressed.npz'
loaded_data = np.load(npz_file)

# Access the array using the key you used to save it (in this case, 'array')
loaded_array = loaded_data['array']
loaded_array

array([[1.     , 0.464  , 0.2134 , ..., 0.1447 , 0.2313 , 0.1211 ],
       [0.4773 , 1.     , 0.1995 , ..., 0.1383 , 0.2267 , 0.1573 ],
       [0.2434 , 0.2103 , 1.     , ..., 0.2136 , 0.2091 , 0.1193 ],
       ...,
       [0.1481 , 0.1198 , 0.1857 , ..., 1.     , 0.1636 , 0.227  ],
       [0.23   , 0.2056 , 0.1761 , ..., 0.1587 , 1.     , 0.0989 ],
       [0.14   , 0.1543 , 0.10394, ..., 0.2405 , 0.11975, 1.     ]],
      dtype=float16)

In [16]:
loaded_data = np.load(npz_file)

# Access the array using the key you used to save it (in this case, 'array')
loaded_array = loaded_data['array']
loaded_array

array([[1.     , 0.3564 , 0.1954 , ..., 0.1009 , 0.0976 , 0.07794],
       [0.3564 , 1.     , 0.1324 , ..., 0.1249 , 0.08734, 0.11884],
       [0.179  , 0.11456, 1.     , ..., 0.1012 , 0.1083 , 0.0991 ],
       ...,
       [0.06744, 0.0923 , 0.0865 , ..., 1.     , 0.4934 , 0.3901 ],
       [0.0692 , 0.0586 , 0.09875, ..., 0.4963 , 1.     , 0.4978 ],
       [0.0604 , 0.102  , 0.1004 , ..., 0.401  , 0.504  , 1.     ]],
      dtype=float16)

In [11]:
import pandas as pd
content_df=pd.read_csv('./data/content_based_data.csv',index_col='anime_id')
# content_df['demographics'].fillna("",inplace=True)
content_df.fillna("unknown",inplace=True)
content_df

,url,title,type,source,episodes,status,aired,duration,rating,score,...,year,studios,genres,themes,demographics,image_url,small_image_url,large_image_url,embed_url,yt_image
anime_id,,,,,,,,,,,,,,,,,,,,,
1,https://myanimelist.net/anime/1/Cowboy_Bebop,Cowboy Bebop,TV,Original,26.0,Finished Airing,"Apr 3, 1998 to Apr 24, 1999",24 min per ep,R - 17+ (violence & profanity),8.75,...,1998.0,Sunrise,Action Award_Winning Sci-Fi,Adult_Cast Space,unknown,https://cdn.myanimelist.net/images/anime/4/196...,https://cdn.myanimelist.net/images/anime/4/196...,https://cdn.myanimelist.net/images/anime/4/196...,https://www.youtube.com/embed/gY5nDXOtv_o?enab...,https://img.youtube.com/vi/gY5nDXOtv_o/maxresd...
5,https://myanimelist.net/anime/5/Cowboy_Bebop__...,Cowboy Bebop: The Movie,Movie,Original,1.0,Finished Airing,"Sep 1, 2001",1 hr 55 min,R - 17+ (violence & profanity),8.38,...,0.0,Bones,Action Sci-Fi,Adult_Cast Space,unknown,https://cdn.myanimelist.net/images/anime/1439/...,https://cdn.myanimelist.net/images/anime/1439/...,https://cdn.myanimelist.net/images/anime/1439/...,unknown,unknown
6,https://myanimelist.net/anime/6/Trigun,Trigun,TV,Manga,26.0,Finished Airing,"Apr 1, 1998 to Sep 30, 1998",24 min per ep,PG-13 - Teens 13 or older,8.22,...,1998.0,Madhouse,Action Adventure Sci-Fi,Adult_Cast,Shounen,https://cdn.myanimelist.net/images/anime/7/203...,https://cdn.myanimelist.net/images/anime/7/203...,https://cdn.myanimelist.net/images/anime/7/203...,https://www.youtube.com/embed/bJVyIXeUznY?enab...,https://img.youtube.com/vi/bJVyIXeUznY/maxresd...
7,https://myanimelist.net/anime/7/Witch_Hunter_R...,Witch Hunter Robin,TV,Original,26.0,Finished Airing,"Jul 3, 2002 to Dec 25, 2002",25 min per ep,PG-13 - Teens 13 or older,7.25,...,2002.0,Sunrise,Action Drama Mystery Supernatural,Detective,unknown,https://cdn.myanimelist.net/images/anime/10/19...,https://cdn.myanimelist.net/images/anime/10/19...,https://cdn.myanimelist.net/images/anime/10/19...,https://www.youtube.com/embed/7UkaILjPk8M?enab...,https://img.youtube.com/vi/7UkaILjPk8M/maxresd...
8,https://myanimelist.net/anime/8/Bouken_Ou_Beet,Beet the Vandel Buster,TV,Manga,52.0,Finished Airing,"Sep 30, 2004 to Sep 29, 2005",23 min per ep,PG - Children,6.99,...,2004.0,Toei Animation,Action Adventure Fantasy,unknown,Shounen,https://cdn.myanimelist.net/images/anime/7/215...,https://cdn.myanimelist.net/images/anime/7/215...,https://cdn.myanimelist.net/images/anime/7/215...,unknown,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56923,https://myanimelist.net/anime/56923/Lv2_kara_C...,Chillin' in Another World with Level 2 Super C...,TV,Light novel,12.0,Currently Airing,"Apr 8, 2024 to ?",24 min per ep,PG-13 - Teens 13 or older,7.02,...,2024.0,J.C.Staff,Adventure Fantasy Romance,Isekai,unknown,https://cdn.myanimelist.net/images/anime/1103/...,https://cdn.myanimelist.net/images/anime/1103/...,https://cdn.myanimelist.net/images/anime/1103/...,https://www.youtube.com/embed/MUn_fypkCfU?enab...,https://img.youtube.com/vi/MUn_fypkCfU/maxresd...
56948,https://myanimelist.net/anime/56948/Boku_no_Ko...,Boku no Kokoro no Yabai Yatsu: Twi-Yaba,ONA,Web manga,1.0,Finished Airing,"Dec 10, 2023",21 min,PG-13 - Teens 13 or older,7.73,...,0.0,Shin-Ei Animation,Comedy Romance,School,Shounen,https://cdn.myanimelist.net/images/anime/1869/...,https://cdn.myanimelist.net/images/anime/1869/...,https://cdn.myanimelist.net/images/anime/1869/...,unknown,unknown
57100,https://myanimelist.net/anime/57100/The_New_Gate,The New Gate,TV,Light novel,unknown,Currently Airing,"Apr 14, 2024 to ?",23 min,PG-13 - Teens 13 or older,6.70,...,2024.0,Yokohama Animation Lab,Action Adventure Fantasy,Isekai,unknown,https://cdn.myanimelist.net/images/anime/1898/...,https://cdn.myanimelist.net/images/anime/1898/...,https://cdn.myanimelist.net/images/anime/1898/...,https://www.youtube.com/embed/3XUJOkFXRiw?enab...,https://img.youtube.com/vi/3XUJOkFXRiw/maxresd...


In [10]:
content_df.iloc[1]

url                https://myanimelist.net/anime/5/Cowboy_Bebop__...
title                                        Cowboy Bebop: The Movie
type                                                           Movie
source                                                      Original
episodes                                                         1.0
status                                               Finished Airing
aired                                                    Sep 1, 2001
duration                                                 1 hr 55 min
rating                                R - 17+ (violence & profanity)
score                                                           8.38
scored_by                                                   215590.0
rank                                                           194.0
popularity                                                       619
favorites                                                       1582
synopsis           Another day, an

In [19]:
(content_df['genres']+content_df['themes']+content_df['demographics']).apply(lambda x:x.replace('unknown','')).apply(lambda x:x.split(' ')).apply(lambda x:list(filter(lambda i:len(i)>0,x)))

['Action', 'Fantasy', 'Sci-Fi', 'Military']

In [21]:
content_df['tags']=(content_df['genres']+content_df['themes']+content_df['demographics']).apply(lambda x:x.replace('unknown','')).apply(lambda x:x.split(' ')).apply(lambda x:list(filter(lambda i:len(i)>0,x)))

In [25]:
tag="Shounen"
content_df[content_df['tags'].apply(lambda x: tag in x)].sort_values(by='popularity',ascending=True)

,url,title,type,source,episodes,status,aired,duration,rating,score,...,studios,genres,themes,demographics,image_url,small_image_url,large_image_url,embed_url,yt_image,tags
anime_id,,,,,,,,,,,,,,,,,,,,,
16498,https://myanimelist.net/anime/16498/Shingeki_n...,Attack on Titan,TV,Manga,25.0,Finished Airing,"Apr 7, 2013 to Sep 29, 2013",24 min per ep,R - 17+ (violence & profanity),8.55,...,Wit Studio,Action Award_Winning Drama Suspense,Gore Military Survival,Shounen,https://cdn.myanimelist.net/images/anime/10/47...,https://cdn.myanimelist.net/images/anime/10/47...,https://cdn.myanimelist.net/images/anime/10/47...,https://www.youtube.com/embed/LHtdKWJdif4?enab...,https://img.youtube.com/vi/LHtdKWJdif4/maxresd...,"[Action, Award_Winning, Drama, Suspense, Gore,..."
1535,https://myanimelist.net/anime/1535/Death_Note,Death Note,TV,Manga,37.0,Finished Airing,"Oct 4, 2006 to Jun 27, 2007",23 min per ep,R - 17+ (violence & profanity),8.62,...,Madhouse,Supernatural Suspense,Psychological,Shounen,https://cdn.myanimelist.net/images/anime/1079/...,https://cdn.myanimelist.net/images/anime/1079/...,https://cdn.myanimelist.net/images/anime/1079/...,https://www.youtube.com/embed/Vt_3c8BgxV4?enab...,https://img.youtube.com/vi/Vt_3c8BgxV4/maxresd...,"[Supernatural, Suspense, Psychological, Shounen]"
5114,https://myanimelist.net/anime/5114/Fullmetal_A...,Fullmetal Alchemist: Brotherhood,TV,Manga,64.0,Finished Airing,"Apr 5, 2009 to Jul 4, 2010",24 min per ep,R - 17+ (violence & profanity),9.09,...,Bones,Action Adventure Drama Fantasy,Military,Shounen,https://cdn.myanimelist.net/images/anime/1208/...,https://cdn.myanimelist.net/images/anime/1208/...,https://cdn.myanimelist.net/images/anime/1208/...,https://www.youtube.com/embed/--IcmZkvL0Q?enab...,https://img.youtube.com/vi/--IcmZkvL0Q/maxresd...,"[Action, Adventure, Drama, Fantasy, Military, ..."
38000,https://myanimelist.net/anime/38000/Kimetsu_no...,Demon Slayer: Kimetsu no Yaiba,TV,Manga,26.0,Finished Airing,"Apr 6, 2019 to Sep 28, 2019",23 min per ep,R - 17+ (violence & profanity),8.47,...,ufotable,Action Award_Winning Fantasy,Historical,Shounen,https://cdn.myanimelist.net/images/anime/1286/...,https://cdn.myanimelist.net/images/anime/1286/...,https://cdn.myanimelist.net/images/anime/1286/...,https://www.youtube.com/embed/6vMuWuWlW4I?enab...,https://img.youtube.com/vi/6vMuWuWlW4I/maxresd...,"[Action, Award_Winning, Fantasy, Historical, S..."
31964,https://myanimelist.net/anime/31964/Boku_no_He...,My Hero Academia,TV,Manga,13.0,Finished Airing,"Apr 3, 2016 to Jun 26, 2016",24 min per ep,PG-13 - Teens 13 or older,7.86,...,Bones,Action,School Super_Power,Shounen,https://cdn.myanimelist.net/images/anime/10/78...,https://cdn.myanimelist.net/images/anime/10/78...,https://cdn.myanimelist.net/images/anime/10/78...,https://www.youtube.com/embed/D5fYOnwYkj4?enab...,https://img.youtube.com/vi/D5fYOnwYkj4/maxresd...,"[Action, School, Super_Power, Shounen]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1736,https://myanimelist.net/anime/1736/BT_X_Neo,B'T X Neo,OVA,Manga,14.0,Finished Airing,"Aug 21, 1997 to Jan 21, 1998",24 min per ep,PG-13 - Teens 13 or older,7.24,...,TMS Entertainment,Action Adventure,unknown,Shounen,https://cdn.myanimelist.net/images/anime/1040/...,https://cdn.myanimelist.net/images/anime/1040/...,https://cdn.myanimelist.net/images/anime/1040/...,unknown,unknown,"[Action, Adventure, Shounen]"
1983,https://myanimelist.net/anime/1983/Black_Jack_...,Black Jack: The Two Doctors of Darkness,Movie,Manga,1.0,Finished Airing,"Dec 17, 2005",1 hr 32 min,PG-13 - Teens 13 or older,7.21,...,Tezuka Productions,Drama,Medical,Shounen,https://cdn.myanimelist.net/images/anime/4/403...,https://cdn.myanimelist.net/images/anime/4/403...,https://cdn.myanimelist.net/images/anime/4/403...,unknown,unknown,"[Drama, Medical, Shounen]"
2122,https://myanimelist.net/anime/2122/Captain_Tsu...,Captain Tsubasa: Saikyou no Teki! Holland Youth,OVA,Manga,1.0,Finished Airing,"Dec 16, 